# HR Analytics

<img src = 'https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/hr_1920x480_s5WuoZs-thumbnail-1200x1200-90.jpg'>

Practice Problem: https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018-1/

## HR Analytics

HR analytics is revolutionising the way human resources departments operate, leading to higher efficiency and better results overall. Human resources has been using analytics for years. However, the collection, processing and analysis of data has been largely manual, and given the nature of human resources dynamics and HR KPIs, the approach has been constraining HR. Therefore, it is surprising that HR departments woke up to the utility of machine learning so late in the game. Here is an opportunity to try predictive analytics in identifying the employees most likely to get promoted.

## Problem Statement

Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion *(only for manager position and below)* and prepare them in time. Currently the process, they are following is:

* They first identify a set of employees based on recommendations/ past performance
* Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
* At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion

For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle. 

<img src = 'https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/09/wns_hack_im_1.jpg'>

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

## Evaluation Metric

The evaluation metric for this competition is F1 Score.

## Public and Private Split

Test data is further randomly divided into Public (40%) and Private (60%) data.

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

## Entorno

In [1]:
import sys
sys.version

'3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]'

In [2]:
!conda info --envs

# conda environments:
#
base                  *  C:\Users\antho\Anaconda3



## Paquetes

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
import random
import warnings
import pickle

warnings.filterwarnings('ignore')


seed = 2020
random.seed(seed)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()

DATA = Path('../../data') 
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'    

MODEL = Path('../../model') 

In [4]:
pd.__version__

'1.1.3'

In [5]:
np.__version__

'1.19.2'

In [6]:
sklearn.__version__

'0.23.2'

## Lectura de datos

In [7]:
os.listdir(f'{PROCESSED}')

['.DS_Store',
 'preprocess_v1_capping_values.pkl',
 'preprocess_v1_impute_values.pkl',
 'preprocess_v1_ohe.pkl',
 'preprocess_v1_ohe_columns.pkl',
 'preprocess_v1_over50_train.csv',
 'preprocess_v1_scaler.pkl',
 'preprocess_v1_train.csv',
 'preprocess_v1_under50_train.csv',
 'preprocess_v1_val.csv',
 'preprocess_v2_capping_values.pkl',
 'preprocess_v2_knnimputation.pkl',
 'preprocess_v2_ohe.pkl',
 'preprocess_v2_ohe_columns.pkl',
 'preprocess_v2_over50_train.csv',
 'preprocess_v2_scaler.pkl',
 'preprocess_v2_scalerimputation.pkl',
 'preprocess_v2_train.csv',
 'preprocess_v2_under50_train.csv',
 'preprocess_v2_val.csv']

In [8]:
preproc_label = 'preprocess_v1'
balanced_label = ''

In [9]:
df_train = pd.read_csv(f'{PROCESSED}/preprocess_v1_train.csv', compression= 'zip')
df_val = pd.read_csv(f'{PROCESSED}/preprocess_v1_val.csv', compression= 'zip')

In [10]:
df_train.shape, df_val.shape

((43846, 61), (10962, 61))

In [11]:
id_columns = 'employee_id'
target = 'is_promoted'

## Features y target

In [12]:
df_train.head()

,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,na_previous_year_rating,na_education,department_Analytics,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_1,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Bachelor's,education_Below Secondary,education_Master's & above,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing,is_promoted
0,-0.431999,0.544625,1.375360,-0.448068,1.356651,-0.153163,-0.328225,-0.284445,-0.212536,-0.32894,-0.220673,-0.21574,-0.137364,1.961378,-0.387219,-0.135368,-0.666307,-0.387881,-0.106419,-0.110615,-0.155332,-0.094979,-0.222429,-0.125042,4.300672,-0.166592,-0.122189,-0.024823,-0.126818,-0.537077,-0.125886,-0.085878,-0.363769,-0.148817,-0.096073,-0.123338,-0.205222,-0.177348,-0.157475,-0.137364,-0.080456,-0.10934,-0.193472,-0.130933,-0.071013,-0.071175,-0.180403,-0.12103,-0.111771,-0.31312,-0.112084,-0.089315,0.634349,-0.120154,-0.612337,-0.651463,0.651463,-1.120159,-0.145991,1.168798,0
1,-0.431999,1.587188,-0.273747,-0.693649,-0.737109,-0.153163,0.270731,-0.284445,-0.212536,-0.32894,-0.220673,-0.21574,-0.137364,-0.509846,2.582520,-0.135368,-0.666307,-0.387881,-0.106419,-0.110615,-0.155332,-0.094979,-0.222429,-0.125042,-0.232522,-0.166592,8.184027,-0.024823,-0.126818,-0.537077,-0.125886,-0.085878,-0.363769,-0.148817,-0.096073,-0.123338,-0.205222,-0.177348,-0.157475,-0.137364,-0.080456,-0.10934,-0.193472,-0.130933,-0.071013,-0.071175,-0.180403,-0.12103,-0.111771,-0.31312,-0.112084,-0.089315,-1.576419,-0.120154,1.633087,1.535007,-1.535007,-1.120159,-0.145991,1.168798,0
2,-0.431999,3.020713,-0.273747,3.481229,-0.737109,-0.153163,0.570210,-0.284445,-0.212536,-0.32894,-0.220673,-0.21574,-0.137364,-0.509846,2.582520,-0.135368,-0.666307,-0.387881,-0.106419,-0.110615,-0.155332,-0.094979,-0.222429,-0.125042,-0.232522,-0.166592,-0.122189,-0.024823,-0.126818,1.861932,-0.125886,-0.085878,-0.363769,-0.148817,-0.096073,-0.123338,-0.205222,-0.177348,-0.157475,-0.137364,-0.080456,-0.10934,-0.193472,-0.130933,-0.071013,-0.071175,-0.180403,-0.12103,-0.111771,-0.31312,-0.112084,-0.089315,0.634349,-0.120154,-0.612337,1.535007,-1.535007,0.892730,-0.145991,-0.855580,0
3,-0.431999,-0.888899,0.550807,-0.448068,-0.737109,-0.153163,-1.076920,-0.284445,-0.212536,-0.32894,-0.220673,-0.21574,-0.137364,-0.509846,-0.387219,-0.135368,1.500810,-0.387881,-0.106419,-0.110615,-0.155332,-0.094979,4.495819,-0.125042,-0.232522,-0.166592,-0.122189,-0.024823,-0.126818,-0.537077,-0.125886,-0.085878,-0.363769,-0.148817,-0.096073,-0.123338,-0.205222,-0.177348,-0.157475,-0.137364,-0.080456,-0.10934,-0.193472,-0.130933,-0.071013,-0.071175,-0.180403,-0.12103,-0.111771,-0.31312,-0.112084,-0.089315,0.634349,-0.120154,-0.612337,-0.651463,0.651463,0.892730,-0.145991,-0.855580,0
4,-0.431999,0.544625,-0.273747,0.779837,-0.737109,-0.153163,1.318905,-0.284445,-0.212536,-0.32894,-0.220673,-0.21574,-0.137364,-0.509846,-0.387219,-0.135368,-0.666307,2.578113,-0.106419,-0.110615,-0.155332,-0.094979,-0.222429,-0.125042,-0.232522,-0.166592,-0.122189,-0.024823,-0.126818,-0.537077,7.943698,-0.085878,-0.363769,-0.148817,-0.096073,-0.123338,-0.205222,-0.177348,-0.157475,-0.137364,-0.080456,-0.10934,-0.193472,-0.130933,-0.071013,-0.071175,-0.180403,-0.12103,-0.111771,-0.31312,-0.112084,-0.089315,-1.576419,-0.120154,1.633087,-0.651463,0

In [13]:
X_train = df_train.drop(target, axis = 1)
y_train = df_train[target]

X_val = df_val.drop(target, axis = 1)
y_val = df_val[target]

In [14]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((43846, 60), (43846,), (10962, 60), (10962,))

## Proporción de target

In [15]:
y_train.mean()

0.08516170232176254

In [16]:
y_val.mean()

0.08520343003101624

## Balanceo de clases (?)

Usar seed para sampling

## Information value de clases

Usar information de R o implementar la función en python (rip)

https://sundarstyles89.medium.com/weight-of-evidence-and-information-value-using-python-6f05072e83eb

## Entrenamiento

Métricas para Grid Search https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

grid search con roc_auc

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [27]:

n_neighbors = list(range(14,25))
p=[1,2]
cv_params = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
knn = KNeighborsClassifier()


gs = GridSearchCV(knn, cv_params, scoring = 'roc_auc', cv = 5, n_jobs= -1)

gs.fit(X_train, y_train)

KeyboardInterrupt: 

In [26]:
pd.DataFrame(gs.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_leaf_size,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,5.267646,0.342894,62.365783,3.598130,29,17,1,"{'leaf_size': 29, 'n_neighbors': 17, 'p': 1}",0.741982,0.741025,0.732198,0.738119,0.728303,0.736325,0.005267,1
34,4.756658,0.078748,60.575381,0.596776,31,17,1,"{'leaf_size': 31, 'n_neighbors': 17, 'p': 1}",0.741982,0.741025,0.732198,0.738119,0.728303,0.736325,0.005267,1
70,5.387221,0.459073,62.092816,4.802066,34,17,1,"{'leaf_size': 34, 'n_neighbors': 17, 'p': 1}",0.741982,0.741025,0.732198,0.738119,0.728303,0.736325,0.005267,1
46,5.131394,0.366657,61.471161,2.283050,32,17,1,"{'leaf_size': 32, 'n_neighbors': 17, 'p': 1}",0.741982,0.741025,0.732198,0.738119,0.728303,0.736325,0.005267,1
22,5.352608,0.631622,65.464204,2.707523,30,17,1,"{'leaf_size': 30, 'n_neighbors': 17, 'p': 1}",0.741982,0.741025,0.732198,0.738119,0.728303,0.736325,0.005267,1
58,5.049117,0.422483,59.866874,1.753940,33,17,1,"{'leaf_size': 33, 'n_neighbors': 17, 'p': 1}",0.741982,0.741025,0.732198,0.738119,0.728303,0.736325,0.005267,1
56,4.894632,0.229919,61.139998,1.054859,33,16,1,"{'leaf_size': 33, 'n_neighbors': 16, 'p': 1}",0.740479,0.740723,0.730101,0.732820,0.724894,0.733803,0.006107,7
8,5.058847,0.149336,61.670175,3.295284,29,16,1,"{'leaf_size': 29, 'n_neighbors': 16, 'p': 1}",0.740479,0.740723,0.730101,0.732820,0.724894,0.733803,0.006107,7
32,4.982165,0.187084,61.088623,2.455848,31,16,1,"{'leaf_size': 31, 'n_neighbors': 16, 'p': 1}",0.740479,0.740723,0.730101,0.732820,0.724894,0.733803,0.006107,7
20,5.860251,0.800986,61.248284,1.398669,30,16,1,"{'leaf_size': 30, 'n_neighbors': 16, 'p': 1}",0.740479,0.740723,0.730101,0.732820,0.724894,0.733803,0.006107,7


In [21]:
gs.best_params_

{'n_neighbors': 14, 'p': 1}

In [22]:
gs.best_score_

0.7286150869228963

In [23]:
best_knn = gs.best_estimator_

In [24]:
best_knn

KNeighborsClassifier(n_neighbors=14, p=1)

Guardar el modelo

In [ ]:
model_label = 'knn_roc_auc'

with open(f'{MODEL}/{model_label}_model.pkl', 'wb') as file:
    pickle.dump(best_dt, file)

## Reentrenamiento con train completo y prueba en validación

In [ ]:
with open(f'{MODEL}/{model_label}_model.pkl', 'rb') as file:
    best_dt = pickle.load(file)

In [ ]:
best_dt.get_params()

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
best_logi.fit(X_train, y_train)

In [ ]:
with open(f'{MODEL}/{model_label}_model_full.pkl', 'wb') as file:
    pickle.dump(best_dt, file)

## Evaluación

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix, precision_recall_curve
from matplotlib import pyplot

In [ ]:
y_train_pred = best_dt.predict(X_train)
y_val_pred = best_dt.predict(X_val)

In [ ]:
y_train_pred

In [ ]:
y_train_pred_proba = best_dt.predict_proba(X_train)[:,1]
y_val_pred_proba = best_dt.predict_proba(X_val)[:,1]

In [ ]:
y_train_pred_proba

In [ ]:
roc_auc_score(y_train, y_train_pred_proba)

In [ ]:
roc_auc_score(y_val, y_val_pred_proba)

In [ ]:
probs = best_dt.predict_proba(X_val)[:, 1]

auc = roc_auc_score(y_val, y_val_pred_proba)
fpr, tpr, umbrales = roc_curve(y_val, y_val_pred_proba)

plt.rcParams.update({'font.size': 9})
plt.plot(fpr, tpr, label= ("Curva ROC") )
plt.xlabel("FPR")
plt.ylabel("TPR (exhaustividad)")
plt.title(("Curva ROC Decision Tree (AUC = %.4f)" % auc))

# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
umbral_05 = np.argmin(np.abs(umbrales - 0.5))
plt.plot(fpr[umbral_05], tpr[umbral_05], 'o', markersize=10,
         label="Umbral 0.5", fillstyle="none", c='r', mew=2)
plt.legend(loc='best', numpoints = 1)
# show the plot
pyplot.show()

In [ ]:
prec, recall, threshold = precision_recall_curve(y_val, probs)

prec_recall = pd.DataFrame({'prec': prec[:-1], 'recall': recall[:-1], 'threshold': threshold})
prec_recall.head()

In [ ]:
prec_recall['f1'] = 2*prec_recall['prec']*prec_recall['recall'] / (prec_recall['prec'] + prec_recall['recall'])

In [ ]:
prec_recall.sort_values(by = 'f1', ascending = False)

In [ ]:
y_val_pred = best_dt.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_val_pred)

precision, recall, umbrales = precision_recall_curve(y_val, probs)

plt.rcParams.update({'font.size': 9})
plt.plot(fpr, tpr, label= ("Precision Recall Curve") )
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.title(("Precision Recall Curce (AUC = %.4f)" % auc))

pyplot.plot([0, 1], [0, 1], linestyle='--')
pyplot.plot(prec, recall, marker='.')

f1 = 2*precision*recall / (precision + recall)
best_f1_index = f1.argmax()
best_umbral = umbrales[best_f1_index]

plt.plot(precision[best_f1_index], recall[best_f1_index], 'o', markersize=10,
         label=f"Mejor umbral: {best_umbral:.2f}", fillstyle="none", c='r', mew=2)
plt.legend(loc='best', numpoints = 1)
pyplot.show()

In [ ]:
labels = np.where(y_val_pred > 0.268443, 1, 0)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_val, labels)

In [ ]:
y_train_pred_label = best_dt.predict(X_train)
y_val_pred_label = best_dt.predict(X_val)

In [ ]:
print(classification_report(y_train, y_train_pred_label))

In [ ]:
print(classification_report(y_val, y_val_pred_label))